<a href="https://colab.research.google.com/drive/1NpHQXk6kqnD5_E3Q596CXtGRaUuizjcB" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



In [1]:
from huggingface_hub import login

login() #login hugging face

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("yelp_polarity")
# Tải tokenizer của mô hình DistilBERT
# Tokenizer này sẽ chuyển từng câu review thành chuỗi số (token IDs) mà mô hình hiểu được.
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Hàm token hóa từng mẫu: chuyển text → token IDs, cắt hoặc đệm cho đủ độ dài
def tokenize(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Áp dụng tokenization cho toàn bộ dataset, xử lý theo batch để tăng tốc
#Tự động xử lý toàn bộ tập dữ liệu: chia nhỏ câu, cắt ngắn, đệm đủ độ dài,…
dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/560000 [00:00<?, ? examples/s]

Map:   0%|          | 0/38000 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

# Tải mô hình DistilBERT đã được huấn luyện sẵn
# và cấu hình cho bài toán phân loại cảm xúc (2 nhãn: tích cực / tiêu cực)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
dataset.save_to_disk("/content/yelp_polarity")


Saving the dataset (0/4 shards):   0%|          | 0/560000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/38000 [00:00<?, ? examples/s]

In [5]:
small_train = dataset["train"].shuffle(seed=42).select(range(1000))
small_eval = dataset["test"].shuffle(seed=42).select(range(1000))

In [6]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # convert the logits to their predicted class
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="yelp_polarity_review_classifier",
    eval_strategy="epoch",
    push_to_hub=True,
)

In [8]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
)
trainer.train()

C:\Users\Admin\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 